In [1]:
import requests
import json
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup
import mp

In [2]:
files = glob('fmt-*.jsonl')
files

['fmt-8.jsonl',
 'fmt-3.jsonl',
 'fmt-1.jsonl',
 'fmt-5.jsonl',
 'fmt-9.jsonl',
 'fmt-0.jsonl',
 'fmt-6.jsonl',
 'fmt-7.jsonl',
 'fmt-4.jsonl',
 'fmt-2.jsonl']

In [3]:
def loop(files):
    files, _ = files
    results = []

    for f in files:
        with open(f) as fopen:
            for l in tqdm(fopen):
                data = json.loads(l)
                url = data['url']
                soup = BeautifulSoup(data['content'], 'lxml')
                title = soup.find_all('h1')[0].text
                p = [p.text for p in soup.find_all('p')]
                try:
                    ul = soup.find_all('ul', {'class': 'td-tags td-post-small-box clearfix'})
                    tags = [li.text for li in ul[0].find_all('li')[1:]]
                except:
                    tags = []
                try:
                    by = soup.find_all('div', {'class': 'td-post-author-name'})[0].text
                    by = by.replace('By ', '').replace('-  ', '').strip()
                except:
                    by = None
                d = {
                    'url': url,
                    'title': title,
                    'p': p,
                    'tags': tags,
                    'by': by,
                }
                results.append(d)
    
    return results

In [4]:
loop((files[:1],0))

2it [00:00, 57.57it/s]


[{'url': 'https://www.freemalaysiatoday.com/category/nation/2019/08/02/johor-sultan-says-intervened-in-arms-procurement-to-stop-absurd-pricing/',
  'title': 'Johor sultan says intervened in arms procurement to stop absurd pricing',
  'p': ['PETALING JAYA: Johor ruler Sultan Ibrahim Sultan Iskandar says he will not hesitate to intervene if irregularities are found in the operations of the Special Forces Regiment, of which he is colonel commandant.',
   'Speaking at the regiment’s 54th anniversary parade at Camp Iskandar in Mersing today, Sultan Ibrahim highlighted what he called a lack of control and scrutiny in equipment procurement.',
   'He gave the example of a company which he said had charged RM5 million for the supply of a Rigid Riding Craft, calling the price exorbitant and unacceptable.',
   '“When I discovered this, I immediately intervened and sourced for a better quality boat at cheaper cost, thereby making a saving for the government,” he said according to a Facebook post o

In [5]:
results = mp.multiprocessing(files, loop, cores = len(files))

1it [00:00,  8.13it/s]
2it [00:00, 14.77it/s]
31803it [20:09, 26.30it/s]
31804it [20:12, 26.22it/s]
31804it [20:13, 26.21it/s]
31804it [20:14, 26.19it/s]
31804it [20:15, 26.18it/s]
31804it [20:24, 25.97it/s]
31804it [20:24, 25.97it/s]
31804it [20:30, 25.84it/s]


In [6]:
len(results)

254434

In [7]:
with open('parsed-fmt.jsonl', 'w') as fopen:
    for r in results:
        fopen.write(f'{json.dumps(r)}\n')

In [8]:
!head -n 1 parsed-fmt.jsonl

{"url": "https://www.freemalaysiatoday.com/category/nation/2019/08/02/johor-sultan-says-intervened-in-arms-procurement-to-stop-absurd-pricing/", "title": "Johor sultan says intervened in arms procurement to stop absurd pricing", "p": ["PETALING JAYA: Johor ruler Sultan Ibrahim Sultan Iskandar says he will not hesitate to intervene if irregularities are found in the operations of the Special Forces Regiment, of which he is colonel commandant.", "Speaking at the regiment\u2019s 54th anniversary parade at Camp Iskandar in Mersing today, Sultan Ibrahim highlighted what he called a lack of control and scrutiny in equipment procurement.", "He gave the example of a company which he said had charged RM5 million for the supply of a Rigid Riding Craft, calling the price exorbitant and unacceptable.", "\u201cWhen I discovered this, I immediately intervened and sourced for a better quality boat at cheaper cost, thereby making a saving for the government,\u201d he said according to a Facebook post 